In [274]:
import json
import tweepy
import time
import config_twit
import jsonpickle
import pandas as pd
import numpy as np
import datetime
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")

In [275]:
timestr = time.strftime("%Y%m%d-%H%M%S")
query = '#kathua OR "kathua" AND "rape"'
max_tweets = 2000

In [276]:
# Enter your keys/secrets as strings in the following fields. config_twit is a .py file that contains 
# my twitter api auth codes 

API_KEY = config_twit.api_key
API_SECRET = config_twit.api_secret
ACCESS_TOKEN = config_twit.ACCESS_TOKEN
ACCESS_TOKEN_SECRET = config_twit.ACCESS_TOKEN_SECRET

# assuming twitter_authentication.py contains each of the 4 oauth elements (1 per line)

auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)


In [277]:
# extract data from tweets that match search terms 

searched_tweets = [status for status in tweepy.Cursor(api.search, q=query).items(max_tweets)]
tweet_id = [s.id for s in searched_tweets]
user_id = [s.user.id for s in searched_tweets]
screen_name = [s.user.screen_name for s in searched_tweets]
followers = [s.user.followers_count for s in searched_tweets]
retweet_count = [s.retweet_count for s in searched_tweets]
locations = [s.user.location for s in searched_tweets]


%time
node:
{"name":"Mme.Burgon","group":7}

link:
{"source":1,"target":0,"value":1}


In [278]:
# df to parse data into network graph - here make df for convenience but later could have lists

df = pd.DataFrame({'tweet_id':tweet_id, 'screen_name':screen_name, 'source':user_id, 'followers':followers, 'locations':locations})
most_followed_tweeters = df.sort_values(by='followers', ascending=False).head(10).drop_duplicates(subset='screen_name')
most_followed_tweets = zip(most_followed_tweeters.tweet_id, most_followed_tweeters.screen_name, 
                           most_followed_tweeters.followers)


> Links list

In [279]:
# make links list: get retweet list of each tweet

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True)

users_retweeted_dict ={}

retweet_lst_names=[]
for status_id, user_id, followers in most_followed_tweets:
    
    retweet_lst=[]
    retweet_lst_name =[]
    
    for retweet_id in api.retweets(status_id):
        retweet_lst.append(retweet_id.user.id)
        retweet_lst_name.append(retweet_id.user.name)
        
    
    users_retweeted_dict[user_id] = retweet_lst_name
    retweet_lst_names.append(retweet_lst_name)

# flatten list
retweet_lst_names = [j for i in retweet_lst_names for j in i]

In [280]:
for j,k,i in most_followed_tweets:
    print(j,k,i)

In [281]:
users_retweeted_dict

{'CNNnews18': ['Hiren Parekh'],
 'EconomicTimes': ['Varun Vijay',
  'Narinder Happa.(नरेन्द्र हप्पा)',
  'Raghav Ohri',
  'Sarthak Chakravarty',
  'Selvakumar Pillai'],
 'IndiaToday': ['Riz Sk',
  'rishabh jain',
  'madhukar kashyap',
  'Badal Singh',
  '🐝மின்மினி🐝',
  'Harshit Dwivedi',
  'Anil Kumar Katiyar',
  'yogesh shinde',
  'vidyut kaji',
  'Santosh',
  'Supriya Bhardwaj',
  'Khupmin Haokip',
  'Ashish Bhardwaj'],
 'TimesNow': ['Mohit $marty....', 'Muhammad Iqbal'],
 'aajtak': ['Sohan Bhadoriya',
  'Ramprasad Lodha ABVP',
  'Himanshu Rana',
  'FIROJ ALI',
  'Mamta sishodia',
  'Shabi Abbas',
  'aamir',
  'arjun singh chauhan 🇮🇳'],
 'htTweets': ['MANISH KUMAR JHA', 'Yugesh'],
 'ndtv': ['Mohammed rafique jun',
  "India's Big Debate",
  'MOHAMMAD DANISH',
  'Anil Dikshit',
  'NIKHIL P BHAGWAT',
  'Bishnu Sahoo',
  'Mithu De'],
 'the_hindu': ['Devender Yadav',
  'Narinder Happa.(नरेन्द्र हप्पा)',
  'Sentil Kumar',
  '@Newsoholic'],
 'timesofindia': ['Varun Vijay',
  'Shweta Tiwary'

In [282]:
# then replace tweet id with index number

links_lst=[]
for k in users_retweeted_dict:

    if users_retweeted_dict[k]:
        for u in users_retweeted_dict[k]:
            links_lst.append({"source":k,"target":u})


In [283]:
users_retweeted_dict

{'CNNnews18': ['Hiren Parekh'],
 'EconomicTimes': ['Varun Vijay',
  'Narinder Happa.(नरेन्द्र हप्पा)',
  'Raghav Ohri',
  'Sarthak Chakravarty',
  'Selvakumar Pillai'],
 'IndiaToday': ['Riz Sk',
  'rishabh jain',
  'madhukar kashyap',
  'Badal Singh',
  '🐝மின்மினி🐝',
  'Harshit Dwivedi',
  'Anil Kumar Katiyar',
  'yogesh shinde',
  'vidyut kaji',
  'Santosh',
  'Supriya Bhardwaj',
  'Khupmin Haokip',
  'Ashish Bhardwaj'],
 'TimesNow': ['Mohit $marty....', 'Muhammad Iqbal'],
 'aajtak': ['Sohan Bhadoriya',
  'Ramprasad Lodha ABVP',
  'Himanshu Rana',
  'FIROJ ALI',
  'Mamta sishodia',
  'Shabi Abbas',
  'aamir',
  'arjun singh chauhan 🇮🇳'],
 'htTweets': ['MANISH KUMAR JHA', 'Yugesh'],
 'ndtv': ['Mohammed rafique jun',
  "India's Big Debate",
  'MOHAMMAD DANISH',
  'Anil Dikshit',
  'NIKHIL P BHAGWAT',
  'Bishnu Sahoo',
  'Mithu De'],
 'the_hindu': ['Devender Yadav',
  'Narinder Happa.(नरेन्द्र हप्पा)',
  'Sentil Kumar',
  '@Newsoholic'],
 'timesofindia': ['Varun Vijay',
  'Shweta Tiwary'

In [284]:
# get followers counts

top_tweeters_follower_counts=[]
mft = most_followed_tweeters.followers

normalized_followers = np.round((np.abs(np.array(mft)-np.array(mft).mean())/mft.std())*10, 0)



count=0

for i in links_lst:
    for j,k in zip(most_followed_tweeters.source, normalized_followers ) :
        
        if i['source'] == j:
            
            top_tweeters_follower_counts.append(int(k))
    
    
# for k,v in zip(most_followed_tweeters.user_id, most_followed_tweeters.followers):
    
    
#    most_followed_tweeters.followers
# links_lst[0]['source']


In [298]:
# put all elements in link list together

count=0
count1=0
links_r =[]
for k,v in users_retweeted_dict.items():
    for y,t in zip(most_followed_tweeters.source, normalized_followers ):
        if y == k:
            value = t
    for i in v:
        links_r.append({"source":k, "target": i, "value": int(value)})
        

In [299]:
links_r

[{'source': 'timesofindia', 'target': 'Varun Vijay', 'value': 12},
 {'source': 'timesofindia', 'target': 'Shweta Tiwary', 'value': 12},
 {'source': 'timesofindia', 'target': 'Braja Bandhu Behera', 'value': 12},
 {'source': 'timesofindia', 'target': 'Mohit $marty....', 'value': 12},
 {'source': 'ndtv', 'target': 'Mohammed rafique jun', 'value': 12},
 {'source': 'ndtv', 'target': "India's Big Debate", 'value': 12},
 {'source': 'ndtv', 'target': 'MOHAMMAD DANISH', 'value': 12},
 {'source': 'ndtv', 'target': 'Anil Dikshit', 'value': 12},
 {'source': 'ndtv', 'target': 'NIKHIL P BHAGWAT', 'value': 12},
 {'source': 'ndtv', 'target': 'Bishnu Sahoo', 'value': 12},
 {'source': 'ndtv', 'target': 'Mithu De', 'value': 12},
 {'source': 'TimesNow', 'target': 'Mohit $marty....', 'value': 12},
 {'source': 'TimesNow', 'target': 'Muhammad Iqbal', 'value': 12},
 {'source': 'aajtak', 'target': 'Sohan Bhadoriya', 'value': 12},
 {'source': 'aajtak', 'target': 'Ramprasad Lodha ABVP', 'value': 12},
 {'source':

>Node list

In [300]:
# node: make "group" from location

location_india =[]
for i in most_followed_tweeters.locations:
    if 'India' or 'New Delhi' in i:
        location_india.append(1)
    else:
        location_india.append(0)
# 

In [301]:
# make nodes lst

nodes_lst = []
for x,y in zip(most_followed_tweeters.screen_name, location_india):
     nodes_lst.append({"id":x, "group":y})

for x,y in enumerate(retweet_lst_names):
     nodes_lst.append({"id":y, "group":2})



> export

In [302]:
# putting it together
data = {}
data['nodes'] = nodes_lst
data['links'] = links_r

In [303]:
import json
with open('network_data_twitter.json', 'w') as outfile:
    json.dump(data, outfile)

In [304]:
data

{'links': [{'source': 'timesofindia', 'target': 'Varun Vijay', 'value': 12},
  {'source': 'timesofindia', 'target': 'Shweta Tiwary', 'value': 12},
  {'source': 'timesofindia', 'target': 'Braja Bandhu Behera', 'value': 12},
  {'source': 'timesofindia', 'target': 'Mohit $marty....', 'value': 12},
  {'source': 'ndtv', 'target': 'Mohammed rafique jun', 'value': 12},
  {'source': 'ndtv', 'target': "India's Big Debate", 'value': 12},
  {'source': 'ndtv', 'target': 'MOHAMMAD DANISH', 'value': 12},
  {'source': 'ndtv', 'target': 'Anil Dikshit', 'value': 12},
  {'source': 'ndtv', 'target': 'NIKHIL P BHAGWAT', 'value': 12},
  {'source': 'ndtv', 'target': 'Bishnu Sahoo', 'value': 12},
  {'source': 'ndtv', 'target': 'Mithu De', 'value': 12},
  {'source': 'TimesNow', 'target': 'Mohit $marty....', 'value': 12},
  {'source': 'TimesNow', 'target': 'Muhammad Iqbal', 'value': 12},
  {'source': 'aajtak', 'target': 'Sohan Bhadoriya', 'value': 12},
  {'source': 'aajtak', 'target': 'Ramprasad Lodha ABVP', '

In [205]:
# build the dataframe

followers = [s.user.followers_count for s in searched_tweets]
friends = [s.user.friends_count for s in searched_tweets]
language = [s.lang for s in searched_tweets]
listed_count = [s.user.listed_count for s in searched_tweets]
locations = [s.user.location for s in searched_tweets]
messages = [s.text.encode('utf8') for s in searched_tweets]
posted_date = [s.created_at for s in searched_tweets]
retweet_count = [s.retweet_count for s in searched_tweets]
screen_name = [s.user.screen_name for s in searched_tweets]
tweet_id = [s.id for s in searched_tweets]
tweet_source = [s.source for s in searched_tweets] 
user_created_at = [s.user.created_at for s in searched_tweets]
user_id = [s.user.id for s in searched_tweets]
user_name = [s.user.name for s in searched_tweets]
geo = [s.geo for s in searched_tweets]

df = pd.DataFrame({
'followers': followers,
'friends': friends,
'language' : language,
'listed_count' : listed_count,
'locations' : locations,
'messages' : messages,
'posted_date' : posted_date,
'retweet_count': retweet_count,
'screen_name' : screen_name,
'tweet_id': tweet_id,
'tweet_source': tweet_source, 
'user_created_at': user_created_at,
'user_id': user_id,
'user_name': user_name,

})

# output to json file for downloading
df.to_csv('general_data_twitter.csv')
# print(df.shape)


In [15]:
small_lst = df[:100]

In [16]:
location_small_lst, message_small_lst = small_lst.locations,small_lst.messages 

In [17]:
coordinates=[]
for index,loc in enumerate(location_small_lst):
    try:
        location = geolocator.geocode(loc)
        coordinates.append({"lat":location.latitude, "lng":location.longitude, "title":message_small_lst[index]})
    except:
        pass
        


In [206]:
df = df.set_index("posted_date")

In [207]:
df = df.resample("1h").sum().fillna(0).rolling(window=3, min_periods=1).mean()

In [208]:
df.reset_index(inplace=True)

In [211]:
# df.groupby(df.index.map(lambda t: t.minute)).mean()


data_roll=[]
data_retweet=[]
for i,j in enumerate(df['posted_date']):
    data_roll.append(j.strftime("%Y %b %d %H %M")) 
    data_retweet.append(round(df.retweet_count[i],0))

In [214]:
brush_df = pd.DataFrame({'date':data_roll,'price':data_retweet})

In [215]:
brush_df.to_csv('brush_data.csv')

In [196]:
with open('brush_data.csv', 'w') as outfile:
    json.dump(data_brush, outfile)

In [202]:
import csv

pd.DataFrame

with open('brush_data.csv','w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(data_brush)

,0
0,"2018 Jul 28 13 00, 3765.0"
1,"2018 Jul 28 14 00, 1882.0"
2,"2018 Jul 28 15 00, 1255.0"
3,"2018 Jul 28 16 00, 0.0"
4,"2018 Jul 28 17 00, 0.0"
5,"2018 Jul 28 18 00, 0.0"
6,"2018 Jul 28 19 00, 335.0"
7,"2018 Jul 28 20 00, 335.0"
8,"2018 Jul 28 21 00, 335.0"
9,"2018 Jul 28 22 00, 0.0"


In [19]:
# pd.read_csv('twitter_data_new.csv',encoding = "ISO-8859-1")

import re
re.sub("[^0-9]", "", df.locations[0])

if not re.search('\d+', value):
    # no numbers
else:
    # numbers present


location_nums=[]
location_other=[]
for i,j in enumerate(df.locations):
    if re.sub("[^0-9]", "", j) != "":
        location_nums.append((i,j))
    else:
        location_other.append((i,j))
    

In [ ]:
from gmplot import gmplot

from IPython.core.display import display, HTML
display(HTML('<h1>Hello, world!</h1>'))


gmap = gmplot.GoogleMapPlotter(37.766956, -122.438481, 13)
# gmap = gmplot.GoogleMapPlotter.from_geocode("San Francisco")

top_attraction_lats, top_attraction_lons = zip(*[
    (37.769901, -122.498331),
    (37.768645, -122.475328),
    (37.771478, -122.468677),
    (37.769867, -122.466102),
    (37.767187, -122.467496),
    (37.770104, -122.470436)
    ])

gmap.scatter(top_attraction_lats, top_attraction_lons, '#3B0B39', size=40, marker=False
             
gmap.draw("my_map.html")